# Projeto 1 - Ciência dos Dados

Nome: Lourenço Meirelles

Nome: Lucca Lima

Nome: Pedro Drumond

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Pedro Drumond\Documents\Insper 2o semestre\CDados\GitHub_CD\Projeto_1_CDados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'Primevideo.xlsx'

In [4]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Classificacao
0,"""eu sou mais eu"" não tem roteiro criativo, par...",2
1,#nineperfectstrangers \nsérie muito boa no @pr...,3
2,#primevideo quero uma sequência pra ontem de ...,2
3,@camila_cabello @cinderella @primevideo amei p...,3
4,@camila_cabello @cinderella @primevideo bravo!👏,3


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Classificação
0,@janainadobrasil esse daqui é a melhor opção p...,2
1,"@camila_cabello @primevideo ficou lindo, te amo",2
2,@camila_cabello @cinderella @primevideo linda!...,3
3,@camila_cabello @cinderella @primevideo ta lin...,3
4,@camilizecabello @mannytsakanika @cinderella @...,2


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O Amazon Prime Video é uma plataforma de streaming semelhante à Netflix, na qual você pode acessar um catálogo de filmes, séries, etc... além de disponibilizar outras coisas como o Amazon Prime Music e entre outros.

Para classificar, consideramos mensagens que elogiassem a plataforma como relevante, e o resto como irrelevante.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
def cleanup(text):
    #import string
    punctuation = '[\n!-.:@?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed
    #MELHORAR
    
#tirar '@'
#talvez 'e', 'que'

In [7]:
#PRECISAMOS DE:

#P(mensagens relevantes | classificadas como relevantes)
#P(mensagens irrelevantes | classificadas como relevantes)
#P(mensagens irrelevantes | classificadas como irrelevantes)
#P(mensagens relevantes | classificadas como irrelevantes)
#Acurácia ?

In [8]:
#DAR UMA OLHADA EM PANDAS
# Carrega o arquivo de texto
# Como ele possui caracteres com acentos, precisamos carregar utilizando o encoding="utf8".
primevideo = pd.read_excel('Primevideo.xlsx')
primevideo.Classificacao = primevideo.Classificacao.astype('int')
primevideo
primevideo_teste = pd.read_excel('Primevideo.xlsx', sheet_name=1)
primevideo_teste

,Teste,Classificação
0,@janainadobrasil esse daqui é a melhor opção p...,2
1,"@camila_cabello @primevideo ficou lindo, te amo",2
2,@camila_cabello @cinderella @primevideo linda!...,3
3,@camila_cabello @cinderella @primevideo ta lin...,3
4,@camilizecabello @mannytsakanika @cinderella @...,2
...,...,...
294,lançamentos do prime video em setembro de 2021...,0
295,"já é tarde demais para dizer que cinderella, d...",1
296,@teamyoutube minha tv (sony) e meu celular (sa...,1
297,@anyborgesreal @dougfrn baseado no caso ritcho...,0


In [9]:
#Criar um DF (DataFrame) para cada classificação

#Filtra as linhas, separando cada clasificação em um DF sem ter aplicado o 'CleanUp'
filtra_linhas = primevideo.Classificacao == 0
primevideo_classfic_0_raw = primevideo.loc[filtra_linhas, 'Treinamento'].reset_index(drop=True)

filtra_linhas = primevideo.Classificacao == 1
primevideo_classfic_1_raw = primevideo.loc[filtra_linhas, 'Treinamento'].reset_index(drop=True)
filtra_linhas = primevideo.Classificacao == 2
primevideo_classfic_2_raw = primevideo.loc[filtra_linhas, 'Treinamento'].reset_index(drop=True)
filtra_linhas = primevideo.Classificacao == 3
primevideo_classfic_3_raw = primevideo.loc[filtra_linhas, 'Treinamento'].reset_index(drop=True)

print(primevideo_classfic_0_raw)

0      @artofesthetic baseado no caso ritchofen, dois...
1      @laressa baseado no caso ritchofen, dois filme...
2      @popzonebr baseado no caso ritchofen, dois fil...
3      @bbb @gshow baseado no caso ritchofen, dois fi...
4      @flamengo @davidluiz_4 baseado no caso ritchof...
                             ...                        
142    @paulaamorim baseado no caso ritchofen, dois f...
143    @eulaurakeller @valescaoficial baseado no caso...
144    @sepcarloslima @acordadebora um crime real, do...
145    @acordadebora um crime real, dois filmes e dua...
146    @rolealeatorio baseado no caso ritchofen, dois...
Name: Treinamento, Length: 147, dtype: object


In [10]:
#CLEANUP - vai precisar do for
#cleanup(primevideo_classfic_0_raw[0].lower()).split()

In [11]:
a = [1, 3]
b = [1, 4]
print(a+b)
c = b.append(a)
print(c)

[1, 3, 1, 4]
None


In [12]:
#For, criando as listas limpas
lista_primevideo_classfic_0 = []
lista_primevideo_classfic_1 = []
lista_primevideo_classfic_2 = []
lista_primevideo_classfic_3 = []

lista_com_todos_tuites_limpos = []

#Cria a lista lista_primevideo_classfic que possui  as classificações de cada categoria
lista_tuites = []
for celula in primevideo_classfic_0_raw:
    
    celula_limpa = cleanup(celula.lower())
    lista_com_todos_tuites_limpos += celula_limpa
    lista_primevideo_classfic_0 += celula_limpa.split()
    #print(lista_primevideo_classfic_0)
    #lista_tuites.append(celula_limpa.split())
    
for celula in primevideo_classfic_1_raw:
    celula_limpa = cleanup(celula.lower())
    lista_com_todos_tuites_limpos += celula_limpa
    lista_primevideo_classfic_1 += celula_limpa.split()
    #print(lista_primevideo_classfic_1)
    #lista_tuites.append(celula_limpa.split())
    
for celula in primevideo_classfic_2_raw:
    celula_limpa = cleanup(celula.lower())
    lista_com_todos_tuites_limpos += celula_limpa
    lista_primevideo_classfic_2 += celula_limpa.split()
    #print(lista_primevideo_classfic_2)
    #lista_tuites.append(celula_limpa.split())
    
for celula in primevideo_classfic_3_raw:
    celula_limpa = cleanup(celula.lower())
    lista_com_todos_tuites_limpos += celula_limpa
    lista_primevideo_classfic_3 += celula_limpa.split()
    #print(lista_primevideo_classfic_3)
    #lista_tuites.append(celula_limpa.split())
    
'''
for celula in primevideo_classfic_3_raw:
    celula_limpa = cleanup(celula.lower()).split()
    lista_primevideo_classfic_3 += celula_limpa
    print(lista_primevideo_classfic_3)
'''



'\nfor celula in primevideo_classfic_3_raw:\n    celula_limpa = cleanup(celula.lower()).split()\n    lista_primevideo_classfic_3 += celula_limpa\n    print(lista_primevideo_classfic_3)\n'

In [13]:
for tuite in primevideo_teste['Teste']:
    celula_limpa = cleanup(celula.lower())
    lista_tuites.append(celula_limpa.split())

In [14]:
#Convertendo as tabelas para pd.Series

classfic_0 = pd.Series(lista_primevideo_classfic_0)
print(classfic_0.value_counts())
print('##############################################')
classfic_1 = pd.Series(lista_primevideo_classfic_1)
print(classfic_1.value_counts())
print('##############################################')
classfic_2 = pd.Series(lista_primevideo_classfic_2)
print(classfic_2.value_counts())
print('##############################################')
classfic_3 = pd.Series(lista_primevideo_classfic_3)
classfic_3.value_counts()


e          115
que         96
pais        82
matou       82
prime       79
          ... 
ser          1
merda        1
filmada      1
anuncia      1
outros       1
Length: 853, dtype: int64
##############################################
prime           96
o               70
primevideo      69
e               68
a               65
                ..
florapaulita     1
considerando     1
gravações        1
ontem            1
querer           1
Length: 1295, dtype: int64
##############################################
primevideo               72
camila_cabello           51
e                        34
de                       34
a                        33
                         ..
mudança                   1
https//tco/c6asm2enpt     1
acaso                     1
👀🔥🔥🔥vem                   1
fk                        1
Length: 616, dtype: int64
##############################################


a                        41
primevideo               33
e                        31
o                        31
prime                    28
                         ..
pela                      1
várias                    1
preacher                  1
algum                     1
https//tco/xsrwtvhbtd     1
Length: 569, dtype: int64

In [15]:
#Tabelas de contagem de ocorrencias em cada tabela, usando o value_counts()
tabela_classific_0 = classfic_0.value_counts()
tabela_classific_1 = classfic_1.value_counts()
tabela_classific_2 = classfic_2.value_counts()
tabela_classific_3 = classfic_3.value_counts()

#Juntando todos os tuítes
todos_tuites = lista_primevideo_classfic_0 + lista_primevideo_classfic_1 + lista_primevideo_classfic_2 + lista_primevideo_classfic_3
#Criando tabela dos tuítes usando o pd.Series
todos_tuites_lista = pd.Series(todos_tuites)
tabela_todos_tuites = todos_tuites_lista.value_counts()
tabela_todos_tuites
#tabela_classific_0

#ACHO QUE VAMOS TER QUE USAR TABELA RELATIVA PORQUE TEMOS MAIS AVALIAÇÕES EM UNS DO QUE OUTROS, VER ISSO!

e                248
prime            225
primevideo       208
a                207
que              198
                ... 
youtube            1
masculinidade      1
amazonbr           1
lixo               1
nam                1
Length: 2552, dtype: int64

In [16]:
tabela_0_relativa = classfic_0.value_counts(True)
tabela_0_relativa

e          0.037916
que        0.031652
pais       0.027036
matou      0.027036
prime      0.026047
             ...   
ser        0.000330
merda      0.000330
filmada    0.000330
anuncia    0.000330
outros     0.000330
Length: 853, dtype: float64

In [17]:
tabela_1_relativa = classfic_1.value_counts(True)
tabela_1_relativa

prime           0.029925
o               0.021820
primevideo      0.021509
e               0.021197
a               0.020262
                  ...   
florapaulita    0.000312
considerando    0.000312
gravações       0.000312
ontem           0.000312
querer          0.000312
Length: 1295, dtype: float64

In [18]:
tabela_2_relativa = classfic_2.value_counts(True)
tabela_2_relativa

primevideo               0.048064
camila_cabello           0.034045
e                        0.022697
de                       0.022697
a                        0.022029
                           ...   
mudança                  0.000668
https//tco/c6asm2enpt    0.000668
acaso                    0.000668
👀🔥🔥🔥vem                  0.000668
fk                       0.000668
Length: 616, dtype: float64

In [19]:
tabela_3_relativa = classfic_3.value_counts(True)
tabela_3_relativa

a                        0.031418
primevideo               0.025287
e                        0.023755
o                        0.023755
prime                    0.021456
                           ...   
pela                     0.000766
várias                   0.000766
preacher                 0.000766
algum                    0.000766
https//tco/xsrwtvhbtd    0.000766
Length: 569, dtype: float64

In [20]:
tuite = '@rolealeatorio baseado no caso ritchofen, dois filmes e duas versões: a menina que matou os pais e o menino que matou meus pais'
tuite = cleanup(tuite.lower())
tuite.split()

['rolealeatorio',
 'baseado',
 'no',
 'caso',
 'ritchofen',
 'dois',
 'filmes',
 'e',
 'duas',
 'versões',
 'a',
 'menina',
 'que',
 'matou',
 'os',
 'pais',
 'e',
 'o',
 'menino',
 'que',
 'matou',
 'meus',
 'pais']

In [21]:
prob_inicial_0 = 1
for palavra in tuite.split():
    prob_0 = tabela_0_relativa[palavra] * prob_inicial_0 #acho que aqui está erraddo
prob_0

0.027035938015166502

In [24]:
lista_tuites #TÁ CERTOOOOOO

[['suymayi',
  'sabia',
  'q',
  'a',
  'versão',
  'original',
  'dessa',
  'série',
  'e',
  'de',
  'israel',
  'ela',
  'tá',
  'na',
  'prime',
  'vídeo',
  'i',
  'e',
  'maravilhosa',
  'os',
  'atores',
  'são',
  'lindos'],
 ['suymayi',
  'sabia',
  'q',
  'a',
  'versão',
  'original',
  'dessa',
  'série',
  'e',
  'de',
  'israel',
  'ela',
  'tá',
  'na',
  'prime',
  'vídeo',
  'i',
  'e',
  'maravilhosa',
  'os',
  'atores',
  'são',
  'lindos'],
 ['suymayi',
  'sabia',
  'q',
  'a',
  'versão',
  'original',
  'dessa',
  'série',
  'e',
  'de',
  'israel',
  'ela',
  'tá',
  'na',
  'prime',
  'vídeo',
  'i',
  'e',
  'maravilhosa',
  'os',
  'atores',
  'são',
  'lindos'],
 ['suymayi',
  'sabia',
  'q',
  'a',
  'versão',
  'original',
  'dessa',
  'série',
  'e',
  'de',
  'israel',
  'ela',
  'tá',
  'na',
  'prime',
  'vídeo',
  'i',
  'e',
  'maravilhosa',
  'os',
  'atores',
  'são',
  'lindos'],
 ['suymayi',
  'sabia',
  'q',
  'a',
  'versão',
  'original',
  'd

    Se P(0|tuite) > P(1|tuite) e P(0|tuite) > P(2|tuite) e P(0|tuite) > P(3|tuite):
        é 0
    Se P(1|tuite) > P(0|tuite) e P(1|tuite) > P(2|tuite) e P(1|tuite) > P(3|tuite):
        é 1
    Se P(2|tuite) > P(0|tuite) e P(2|tuite) > P(1|tuite) e P(2|tuite) > P(3|tuite):
        é 2
    Se P(3|tuite) > P(0|tuite) e P(3|tuite) > P(1|tuite) e P(3|tuite) > P(2|tuite):
        é e

In [ ]:
'''PRECISAMOS FAZER:

- La Place
- Comparar as classificações
- Análise crítica da da performance


-LA PLACE:

    -temos que tirar tudo de relativo
    -quantidade de palavras na categoria + 1 / quantidade de palavras na categoria + quantidade de palavras ao todo


'''

In [23]:
classificacao_final = []
for tuite in lista_tuites:
    prob_0 = 1
    prob_1 = 1
    prob_2 = 1
    prob_3 = 1
    for palavra in tuite:
        
        prob_0 *= tabela_0_relativa[palavra]
        prob_1 *= tabela_1_relativa[palavra]
        prob_2 *= tabela_2_relativa[palavra]
        prob_3 *= tabela_3_relativa[palavra]
        
            
        if prob_0 > prob_1 and prob_0 > prob_2 and prob_0 > prob_3:
            print(prob_0)
            classificacao_final.append(0)
            #adicionar na coluna no excel
            
        
        elif prob_1 > prob_0 and prob_1 > prob_2 and prob_1 > prob_3:
            print(prob_1)
            classificacao_final.append(1)
            
        elif prob_2 > prob_0 and prob_2 > prob_1 and prob_2 > prob_3:
            print(prob_2)
            classificacao_final.append(2)
            
        elif prob_3 > prob_0 and prob_3 > prob_1 and prob_3 > prob_1:
            print(prob_3)
            classificacao_final.append(3)
            

    
    
        

SyntaxError: 'return' outside function (<ipython-input-23-2d8558de144d>, line 24)

In [ ]:
#Começando classificador:
'''
def funcao_classificadora(tuite):
    
    prob_inicial_0 = 1
    prob_inicial_1 = 1
    prob_inicial_2 = 1
    prob_inicial_3 = 1
    
    for palavra in tuite.split():
        if tabela_0_relativa[palavra] > 0:
            prob_0 = tabela_0_relativa[palavra] * prob_inicial_0
        if tabela_1_relativa[palavra] > 0:
            prob_1 = tabela_1_relativa[palavra] * prob_inicial_1
        if tabela_2_relativa[palavra] > 0:
            prob_2 = tabela_2_relativa[palavra] * prob_inicial_2
        if tabela_3_relativa[palavra] > 0:
            prob_3 = tabela_3_relativa[palavra] * prob_inicial_3
        else:
            return 'não há essa palavra na database'
            
    if prob_0 > prob_1 and prob_0 > prob_2 and prob_0 > prob_3:
        print(prob_0)
        return 0
    elif prob_1 > prob_0 and prob_1 > prob_2 and prob_1 > prob_3:
        print(prob_1)
        return 1
    elif prob_2 > prob_0 and prob_2 > prob_1 and prob_2 > prob_3:
        print(prob_2)
        return 2
    elif prob_3 > prob_0 and prob_3 > prob_1 and prob_3 > prob_1:
        print(prob_3)
        return 3

print(funcao_classificadora(tuite))
''' 

In [ ]:
#Tem que fazer para cada linha 
#CLASSIFICADOR#

#passa por cada tuíte
for tuite in todos_tuites.Treinamento:
    #P(tuite|classificacao)
    P_tuite_dado_classificacao_0 = todos_tuites_relativa[tuite] * 
    

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**